In [1]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
import seaborn as sns

In [24]:
topic_size = 50
document_size = 1
vocabulary_size = 102660
topic_concentration = 0.01
document_concentration = 0.01

Initialize $\lambda$ obeys gamma distribution, then calculate $E(log\beta)$

In [25]:
def diriExpectationVector(gamma):
    expElogtheta = np.zeros(shape = gamma.shape)
    rowSum = gamma.sum()
    rowDigamma = scipy.special.digamma(rowSum)
    print(gamma.shape[0])
    for i in range (0, gamma.shape[0]):
        expElogtheta[i] = np.exp(scipy.special.digamma(gamma[i]) - rowDigamma)  
    return expElogtheta

In [2]:
scipy.special.digamma(1)

-0.57721566490153309

In [26]:
def diriExpectationMatrix(lambd):
    expElogbeta = np.zeros(shape = lambd.shape)
    for i in range (0, lambd.shape[0]):
        rowSum = lambd[i].sum()
        rowDigamma = scipy.special.digamma(rowSum)
        for j in range (0, lambd.shape[1]):
            x = scipy.special.digamma(lambd[i][j])
            expElogbeta[i][j] = np.exp(x - rowDigamma)  
    return expElogbeta

In [32]:
def diriExpectationPartMatrix(lambd):
    part_expElogbeta = np.zeros(shape = lambd.shape)
    for i in range (0, lambd.shape[0]):
        rowSum = lambd[i].sum()
        rowDigamma = scipy.special.digamma(rowSum)
        for j in wids:
            x = scipy.special.digamma(lambd[i][j])
            part_expElogbeta[i][j] = np.exp(x - rowDigamma)
                
%lprun -f diriExpectationPartMatrix diriExpectationPartMatrix(lambd)        

*** KeyboardInterrupt exception caught in code being profiled.

In [29]:
np.random.seed(0)
lambd = np.random.gamma(100, 1.0/100, (topic_size, vocabulary_size))
expElogbeta = diriExpectationMatrix(lambd)

In [16]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [22]:
%lprun -f diriExpectationMatrix diriExpectationMatrix(lambd)

128


In [8]:
# sns.distplot(lambd[0])
# plt.show()

Read document d to calculate $E(log\theta_d)$

In [2]:
stat = np.zeros(shape = (topic_size, vocabulary_size))
for line in open("./small10Samples.txt"):
    # initialize gamma
    gamma = np.random.gamma(100, 1.0/100, (topic_size))
    meanGammaChange = 2.0
    
    # get words ids 
    line=line.strip()
    pairs = line.split(' ')
    pairs = pairs[1:]
    wids = []
    wcount=[]
    for pair in pairs:
        pair=pair.split(':')
        pair[0]=int(pair[0])
        pair[1]=int(pair[1])
        wids.append(pair[0])
        wcount.append(pair[1])
    phi = np.zeros(shape=(topic_size, len(wids)))
    while meanGammaChange > 1e-3:
        # save last gamma
        lastGamma = np.zeros(shape=(topic_size))
        for i in range (0, topic_size):
            lastGamma[i]=gamma[i]
    
        # change gamma and phi
        expElogtheta = diriExpectationVector(gamma)
        for k in range(0, topic_size):
            for w in range(0, len(wids)):
                phi[k,w] = expElogtheta[k]*expElogbeta[k,wids[w]]
        for k in range(0, topic_size):
            delta = 0.0
            for w in range(0, len(wids)):
                delta = delta + wcount[w]*phi[k,w]
            gamma[k] = topic_concentration + delta
        # calculate change
        meanGammaChange = np.absolute(gamma - lastGamma).sum()/topic_size
        print(meanGammaChange)
        
        

NameError: name 'np' is not defined